In [172]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest,chi2

In [173]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [174]:
## setting work directory

In [175]:
os.chdir("U:\Propensity_Model\Model")

In [176]:
### Loading the required files

In [177]:
pickle_in = open("Final_Data_.pickle","rb")

Final_Data = pickle.load(pickle_in)

pickle_in.close()

In [178]:
### Split data into Train, Test and Validation

In [179]:
x = Final_Data.drop('AUDIT_STATUS',axis=1)
y = pd.DataFrame(Final_Data['AUDIT_STATUS'])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [180]:
## Checking the Selection %

In [181]:
print(y_train['AUDIT_STATUS'].value_counts()/len(y_train)*100) #1.86
print(y_test['AUDIT_STATUS'].value_counts()/len(y_test)*100)   #1.50

0    98.226685
1     1.773315
Name: AUDIT_STATUS, dtype: float64
0    98.119926
1     1.880074
Name: AUDIT_STATUS, dtype: float64


In [182]:
####Data PreProcessing (numeric columns of train_data)

##Separating out Numeric columns from train data

In [183]:
train_num = x_train.select_dtypes(include='number')

In [184]:
##PreProcessing numeric data

In [185]:
sc = StandardScaler()
train_num_process = pd.DataFrame(sc.fit_transform(train_num))
train_num_process.columns = train_num.columns
train_num_process.index = train_num.index

In [186]:
### Combining the processed data to train_data

In [187]:
col_drop = train_num.columns
x_train = x_train.drop(col_drop,axis=1)

In [188]:
## Combining the PreProcessed data to train_data

In [189]:
x_train = pd.concat([x_train,train_num_process],axis=1)

In [190]:
x_train = x_train.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [191]:
## performing one-hot-encoding on categorical features

In [192]:
ctgrl_col = pd.DataFrame(x_train[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_train = x_train.drop(ctgrl_col.columns,axis=1)
x_train = pd.concat([x_train,ctgrl_cols],axis=1)

In [193]:
### Data balancing using SMOTE

In [194]:
sm = SMOTE(sampling_strategy=0.43)

In [195]:
x_sm,y_sm = sm.fit_resample(x_train.values,y_train.values.ravel())

In [196]:
x_sm = pd.DataFrame(x_sm)
y_sm = pd.DataFrame(y_sm)

In [197]:
x_sm.columns = x_train.columns
y_sm.columns = y_train.columns

In [198]:
y_sm.value_counts(normalize=True)*100

AUDIT_STATUS
0               69.931045
1               30.068955
dtype: float64

In [199]:
y_train.value_counts(normalize=True)*100

AUDIT_STATUS
0               98.226685
1                1.773315
dtype: float64

In [200]:
####################################################
####################################################
####################################################

### Model Building

In [152]:
RF_v1 = RandomForestClassifier()


In [31]:
data = {
    'n_estimators':[100,200,300,400,500],
    'criterion':['gini','entropy'],
    'max_depth':[5,10,15,20,25,30],
    'min_samples_split':[1,3,5,7,9,10,12,14],
    'min_samples_leaf':[1,3,5,7,9,10,12,14],
    'max_features':['auto','sqrt','log2']
}

In [34]:
from sklearn.model_selection import RandomizedSearchCV
gscv = RandomizedSearchCV(RF_v1,data,verbose=True,n_jobs=-1)
gscv.fit(x_sm.values,y_sm.values.ravel())

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\User

RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 3, 5, 7, 9, 10,
                                                             12, 14],
                                        'min_samples_split': [1, 3, 5, 7, 9, 10,
                                                              12, 14],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   verbose=True)

In [35]:
gscv.best_params_

{'n_estimators': 200,
 'min_samples_split': 3,
 'min_samples_leaf': 3,
 'max_features': 'sqrt',
 'max_depth': 30,
 'criterion': 'gini'}

In [36]:
gscv.best_score_

0.985460430320711

In [201]:
RF_v1 = RandomForestClassifier(n_estimators=200,min_samples_split=3,min_samples_leaf=3,max_features='sqrt',max_depth=30,criterion='gini')
RF_v1.fit(x_sm.values,y_sm.values.ravel())

RandomForestClassifier(max_depth=30, max_features='sqrt', min_samples_leaf=3,
                       min_samples_split=3, n_estimators=200)

In [202]:
RF_v1.feature_importances_

array([1.94011928e-03, 7.87169477e-03, 4.51460103e-03, 8.01883626e-03,
       5.02729074e-03, 4.31603187e-03, 3.06449128e-03, 1.10131449e-02,
       3.49000186e-03, 3.52570873e-03, 4.86905763e-03, 1.54899342e-03,
       7.79509681e-03, 2.53920082e-03, 3.25784292e-03, 4.09148541e-03,
       5.17678455e-03, 2.10635303e-03, 3.74012668e-03, 3.39625887e-03,
       5.06147894e-03, 7.07238800e-03, 1.00993811e-02, 4.11569845e-03,
       2.37881077e-03, 1.76644275e-03, 2.14244556e-03, 5.13802816e-03,
       2.18765517e-03, 4.23593204e-03, 1.53694694e-02, 1.85480458e-02,
       9.21966697e-04, 0.00000000e+00, 2.63523548e-03, 1.59473194e-03,
       9.37079950e-04, 2.27634757e-04, 5.97286070e-04, 1.88311364e-03,
       0.00000000e+00, 4.50886312e-02, 1.87179713e-02, 1.65935394e-02,
       2.16883452e-02, 1.69910184e-02, 6.37660361e-04, 2.39948591e-02,
       2.24446436e-02, 1.57236485e-02, 1.72350691e-03, 5.74352569e-03,
       8.61307651e-03, 1.67159694e-02, 1.98384490e-02, 1.07329045e-02,
      

In [155]:
plt.figure(figsize=(30,30))
feat_importances = pd.Series(RF_v1.feature_importances_,index=x_sm.columns)
#feat_importances.plot(kind='barh')
print(feat_importances)

REV_MISSING                       0.001761
REV_0636                          0.007773
REV_0301                          0.004864
REV_0300                          0.009837
REV_0250                          0.006010
REV_0305                          0.004639
REV_0637                          0.003305
REV_0260                          0.009507
REV_0450                          0.003555
REV_0306                          0.003164
REV_0258                          0.005309
REV_0420                          0.001848
REV_0272                          0.007615
REV_0730                          0.002833
REV_0762                          0.003497
REV_OTHERS                        0.003709
CPT_MISSING                       0.005453
CPT_A9270                         0.002009
CPT_85025                         0.003817
CPT_80053                         0.002932
CPT_36415                         0.004949
CPT_96375                         0.006237
CPT_96372                         0.010435
CPT_80048  

<Figure size 2160x2160 with 0 Axes>

In [156]:
feat_importances.to_csv(r'U:/Propensity_Model/FEAT.csv',index=True,header=True)

In [383]:
x_sm.dtypes

REV_MISSING                       float64
REV_0636                          float64
REV_0301                          float64
REV_0300                          float64
REV_0250                          float64
REV_0305                          float64
REV_0637                          float64
REV_0260                          float64
REV_0450                          float64
REV_0306                          float64
REV_0258                          float64
REV_0420                          float64
REV_0272                          float64
REV_0730                          float64
REV_0762                          float64
REV_OTHERS                        float64
CPT_MISSING                       float64
CPT_A9270                         float64
CPT_85025                         float64
CPT_80053                         float64
CPT_36415                         float64
CPT_96375                         float64
CPT_96372                         float64
CPT_80048                         

In [203]:
x_sm = pd.DataFrame(x_sm[['OVR_00','REJ_OTHERS','MBR_ST_OTHERS','OVR_27','CHRG','BEN_AMT','ALW_PER_UN','NET_PAID_AMT','UN','DIAG_CNT','VCH_H','MBR_AGE','MOD_CNT','TOB_851',
        'LINE_ITEM_Original, Backed Out','REJ_MISSING','MBR_RSP','REJ_0DC','NTWK_OTHERS','CAS_CAUSE_GENERAL ILLNESS','CLI_58','LOB_3I','LINE_ITEM_Current, Original','CLI_57',
        'OVR_37','SERV_DAYS','CPT_96372']])

In [216]:
x_sm = pd.DataFrame(x_sm[['ADMIT_DX_OTHERS','MBR_ST_FL','TOB_131','REV_0300','VCH_K','SERV_DAYS','LOB_32','CPT_96375','REV_0260',
                         'TOB_851','COPAY','NTWK_OTHERS','LINE_ITEM_Original, Backed Out','LINE_ITEM_Current, Original',
                         'LOB_3I','CAS_CAUSE_GENERAL ILLNESS','MOD_CNT','VCH_H','REJ_MISSING','MBR_AGE','CPT_96372','UN',
                         'OVR_27','CHRG','REJ_0DC','DIAG_CNT','ALLW','MBR_ST_OTHERS','OVR_OTHERS','REJ_OTHERS']])

In [33]:
x_sm = pd.DataFrame(x_sm[['COPAY','TOB_851','LINE_ITEM_Original, Backed Out','LINE_ITEM_Current, Original','LOB_3I',
                         'CAS_CAUSE_GENERAL ILLNESS','MOD_CNT','VCH_H','MBR_AGE','UN','REJ_MISSING','REJ_0DC','CPT_96372',
                         'OVR_27','CHRG','ALLW','DIAG_CNT','OVR_OTHERS','MBR_ST_OTHERS','REJ_OTHERS']])

In [335]:
x_sm = pd.DataFrame(x_sm[['UN','REJ_MISSING','OVR_27','CHRG','REJ_0DC','DIAG_CNT','ALLW','MBR_ST_OTHERS','OVR_OTHERS','REJ_OTHERS']])

In [204]:
RF_v1 = RandomForestClassifier(n_estimators=200,min_samples_split=3,min_samples_leaf=3,max_features='sqrt',max_depth=30,criterion='gini')
RF_v1.fit(x_sm.values,y_sm.values.ravel())

RandomForestClassifier(max_depth=30, max_features='sqrt', min_samples_leaf=3,
                       min_samples_split=3, n_estimators=200)

In [205]:
#####################################

####Data PreProcessing (numeric columns of test_data)

##Separating out Numeric columns from test data

In [206]:
test_num = x_test.select_dtypes(include='number')

In [207]:
##PreProcessing numeric data

In [208]:
test_num_process = pd.DataFrame(sc.transform(test_num))
test_num_process.columns = test_num.columns
test_num_process.index = test_num.index

In [209]:
###########
### Combining the processed data to test_data

In [210]:
col_drop = test_num.columns
x_test = x_test.drop(col_drop,axis=1)

In [211]:
## Combining the Preprocessed data to test_data

In [212]:
x_test = pd.concat([x_test,test_num_process],axis=1)

In [213]:
x_test = x_test.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [214]:
##one-hot-encoding on categorical features

In [215]:
ctgrl_col = pd.DataFrame(x_test[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_test = x_test.drop(ctgrl_col.columns,axis=1)
x_test = pd.concat([x_test,ctgrl_cols],axis=1)


In [216]:
x_test = pd.DataFrame(x_test[['OVR_00','REJ_OTHERS','MBR_ST_OTHERS','OVR_27','CHRG','BEN_AMT','ALW_PER_UN','NET_PAID_AMT','UN','DIAG_CNT','VCH_H','MBR_AGE','MOD_CNT','TOB_851',
        'LINE_ITEM_Original, Backed Out','REJ_MISSING','MBR_RSP','REJ_0DC','NTWK_OTHERS','CAS_CAUSE_GENERAL ILLNESS','CLI_58','LOB_3I','LINE_ITEM_Current, Original','CLI_57',
        'OVR_37','SERV_DAYS','CPT_96372']])

In [138]:
x_test = pd.DataFrame(x_test[['REJ_OTHERS','MBR_ST_OTHERS','OVR_OTHERS','ALLW','DIAG_CNT','CHRG','REJ_0DC','OVR_27','UN','REJ_MISSING','MBR_AGE','CPT_96372'
                    ,'MOD_CNT','VCH_H','CAS_CAUSE_GENERAL ILLNESS','TOB_851','COPAY','LOB_3I','LINE_ITEM_Original, Backed Out','LOB_32','LINE_ITEM_Current, Original'
                    ,'REV_0260','CPT_96375','NTWK_OTHERS','VCH_K']])

In [45]:
x_test = pd.DataFrame(x_test[['ADMIT_DX_OTHERS','MBR_ST_FL','TOB_131','REV_0300','VCH_K','SERV_DAYS','LOB_32','CPT_96375','REV_0260',
                         'TOB_851','COPAY','NTWK_OTHERS','LINE_ITEM_Original, Backed Out','LINE_ITEM_Current, Original',
                         'LOB_3I','CAS_CAUSE_GENERAL ILLNESS','MOD_CNT','VCH_H','REJ_MISSING','MBR_AGE','CPT_96372','UN',
                         'OVR_27','CHRG','REJ_0DC','DIAG_CNT','ALLW','MBR_ST_OTHERS','OVR_OTHERS','REJ_OTHERS']])

In [46]:
x_test = pd.DataFrame(x_test[['COPAY','TOB_851','LINE_ITEM_Original, Backed Out','LINE_ITEM_Current, Original','LOB_3I',
                         'CAS_CAUSE_GENERAL ILLNESS','MOD_CNT','VCH_H','MBR_AGE','UN','REJ_MISSING','REJ_0DC','CPT_96372',
                         'OVR_27','CHRG','ALLW','DIAG_CNT','OVR_OTHERS','MBR_ST_OTHERS','REJ_OTHERS']])

In [347]:
x_test = pd.DataFrame(x_test[['UN','REJ_MISSING','OVR_27','CHRG','REJ_0DC','DIAG_CNT','ALLW','MBR_ST_OTHERS','OVR_OTHERS','REJ_OTHERS']])

In [348]:
##################
### Predicting on the test data

In [349]:
##RF_v1

In [222]:
y_pred = RF_v1.predict(x_test)
y_pred = (RF_v1.predict_proba(x_test)[:,1]>0.2).astype('int').astype('str')
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.9427431989745051
[[6587  302]
 [ 100   32]]
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      6889
           1       0.10      0.24      0.14       132

    accuracy                           0.94      7021
   macro avg       0.54      0.60      0.55      7021
weighted avg       0.97      0.94      0.95      7021



In [221]:
y_pred = RF_v1.predict(x_sm)
y_pred = (RF_v1.predict_proba(x_sm)[:,1]>0.3).astype('int').astype('str')
print(accuracy_score(y_sm,y_pred))
print(confusion_matrix(y_sm,y_pred))

print(classification_report(y_sm,y_pred))

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.9940931906910714
[[27426   159]
 [   74 11787]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     27585
           1       0.99      0.99      0.99     11861

    accuracy                           0.99     39446
   macro avg       0.99      0.99      0.99     39446
weighted avg       0.99      0.99      0.99     39446

